In [1]:
import MetaTrader5 as mt5
import pandas as pd 
mt5.initialize()

True

In [2]:
formato_numerico = "{0:10.2f}"
ativo = list(())
AtivoSymbolInfo = list(())
matriz = list(())
global matriz

In [3]:
def imprimirInfoAtivo(t, y, n):
    print(t, y, formato_numerico.format(n))

In [4]:
# ****************************************************
def RolarOpcoes(ativo):
    n = len(ativo)
    AtivoSymbolInfo.clear()
    rows,cols = (n,9)
    matriz = [[0 for i in range(cols)] for j in range(rows)]
    
    for i in range(n):
        AtivoSymbolInfo.append(mt5.symbol_info(ativo[i]))
        xx = AtivoSymbolInfo[i]
        matriz[i][0] = xx.name
        matriz[i][1] = xx.option_strike
        matriz[i][4] = xx.last
 
        # somente para linha 1, que é da opção vendida, que preciso rolar
        # pega o ASK... mas se não tiver ninguém oferecendo, pega o LAST
        if i == 1:
            matriz[1][3] = xx.ask
            if matriz[1][3] == 0:
                matriz[i][3] = input('Entre o valor do Ask ' + xx.name + ': ')
            gasto = matriz[1][3] * -1
#            if matriz[1][3] == 0:       gasto = matriz[1][4]
 
        # para as linhas 2,3,... ou seja, para as opções dos meses seguintes que poderei vender
        if i>1:
            matriz[i][2] = xx.bid
            if matriz[i][2] == 0:
                matriz[i][2] = input('Entre o valor do Bid ' + xx.name + ': ')
            matriz[i][5] = matriz[i][1] - matriz[1][1] # aumento em R$ strike nova - strike vendida
            matriz[i][6] = (matriz[i][1]/matriz[1][1] - 1) * 100  # % strike nova - strike vendido
            matriz[i][7] = matriz[i][5] + gasto + matriz[i][2] # dif.strike - gasto
            matriz[i][8] = matriz[i][7] / matriz[1][1] * 100

    '''for l in range(n):
        for c in range(9):
            print(f'|{matriz[l][c]:^10}|',end='')
        print()
    '''
    df = pd.DataFrame(matriz, columns=['Ativo','Strike','bid','ask','Last','▲ R$','▲ % Strike','Somatório','% sobre '+matriz[1][0]])
    print(df)
    df.to_excel('ativo.xlsx')

In [11]:
ativo = ['CMIG4', 'CMIGG154', 'CMIGH139'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CMIG4    0.00  0.00  0.00  13.10  0.00    0.000000       0.00   
1  CMIGG154   13.35  0.00  0.46   0.43  0.00    0.000000       0.00   
2  CMIGH139   14.00  0.47  0.00   0.62  0.65    4.868914       0.66   

   % sobre CMIGG154  
0           0.00000  
1           0.00000  
2           4.94382  


In [14]:
ativo = ['ITSA4', 'ITSAG113', 'ITSAH115'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     ITSA4    0.00  0.00  0.00  11.65  0.00    0.000000        0.0   
1  ITSAG113   11.26  0.00  0.69   0.63  0.00    0.000000        0.0   
2  ITSAH115   11.54  0.11  0.00   0.68  0.28    2.486679       -0.3   

   % sobre ITSAG113  
0          0.000000  
1          0.000000  
2         -2.664298  


In [15]:
ativo = ['CSAN3', 'CSANG100', 'CSANH250'] 
RolarOpcoes(ativo)

TypeError: unsupported operand type(s) for +: 'float' and 'str'